<a href="https://colab.research.google.com/github/sam2005-max/t_t_s/blob/main/TTS_using_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 879.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2

In [ ]:
!pip install librosa soundfile


In [ ]:
import torch
import soundfile as sf
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

# Model va vocoderni yuklash
model_name = "Masternlp/speecht5_feniks"
model = SpeechT5ForTextToSpeech.from_pretrained(model_name)
processor = SpeechT5Processor.from_pretrained(model_name)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Speaker embedding yaratish (zarur hollarda o'zgartiriladi)
speaker_embedding = torch.zeros(1, 512)  # Changed to 2 dimensions

def text_to_speech(text):
    # Matnni tokenlarga aylantirish
    inputs = processor(text=text, return_tensors="pt", padding=True)

    # Modeldan ovozli chiqishni generatsiya qilish
    with torch.no_grad():
        speech = model.generate_speech(
            inputs["input_ids"], vocoder=vocoder, speaker_embeddings=speaker_embedding
        )

    # Ovozli chiqishni .wav formatida saqlash
    wav_file = "output.wav"
    sf.write(wav_file, speech.numpy(), 16000)  # 16 kHz chastotada audio saqlash

    # Ovozli chiqishni .mp3 formatida saqlash uchun avval wav formatiga aylantiramiz
    mp3_file = "test_audio5.mp3"
    sf.write(mp3_file, speech.numpy(), 16000, format='MP3')

    print(f"Audio fayl saqlandi: {mp3_file}")

# O'zbekcha matnni kiriting
uzbek_text = "Assalomu aleykum, Man Euphoria kompaniyasidan bosh urolog Javlonbek bo'laman. Okajon sizzi qanaqa muammola bezovta qivotti nimadan shikoyatiz bor, Prostatani shamollashidami Olatning hajimidami yoki Jinsiy zaiflik, menga hamma ma'lumotlarni ochiq oydin eturin man sizzi davolanishizga yordam beraman okajon"
text_to_speech(uzbek_text)

Audio fayl saqlandi: test_audio5.mp3


In [ ]:
speaker_embedding = torch.zeros(1, 512)  # Changed to 2 dimensions

In [ ]:
import gradio as gr
import torch
import numpy as np
import soundfile as sf
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

# Model va vocoderni yuklash
model_name = "Masternlp/Phoenix"
model = SpeechT5ForTextToSpeech.from_pretrained(model_name)
processor = SpeechT5Processor.from_pretrained(model_name)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Speaker embedding yaratish (zarur hollarda o'zgartiriladi)
speaker_embedding = torch.zeros(1, 1, 512)  # Default speaker embedding

def text_to_speech(text):
    # Matnni ovozli chiqishga tayyorlash
    inputs = processor(text=text, return_tensors="pt")

    # Modeldan ovozli chiqishni generatsiya qilish
    with torch.no_grad():
        speech = model.generate_speech(
            inputs["input_values"], vocoder=vocoder, speaker_embeddings=speaker_embedding
        )

    # Ovozli chiqishni .wav formatida saqlash
    audio_file = "output.wav"
    sf.write(audio_file, speech.numpy(), 16000)  # 16 kHz chastotada audio saqlash

    return audio_file

# Gradio interfeysi yaratish
iface = gr.Interface(
    fn=text_to_speech,
    inputs="text",
    outputs="audio",
    title="Uzbek TTS",
    description="O'zbekcha matnni ovozga aylantiruvchi model."
)

# Interfeysni ishga tushirish
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://785a7ed6771f3a0551.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import torch
import numpy as np
import soundfile as sf
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor, SpeechT5HifiGan

# Model va vocoderni yuklash
model_name = "Masternlp/Phoenix"
model = SpeechT5ForTextToSpeech.from_pretrained(model_name)
processor = SpeechT5Processor.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [ ]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speaker_embedding = torch.zeros(1, 1, 512)  # Odatiy speaker embedding (mos yozuvchiga qarab yangilanadi)

def text_to_speech(text):
    inputs = processor(text=text, return_tensors="pt")

    with torch.no_grad():
        speech = model.generate_speech(
            inputs["input_values"], vocoder=vocoder, speaker_embeddings=speaker_embedding
        )

    # Ovozli chiqishni .wav formatida saqlash
    audio_file = "output.mp3"
    sf.write(audio_file, speech.numpy(), 16000)  # 16000 Hz chastotada audio saqlash

    return audio_file

# Gradio interfeysiearths I am going to
iface = gr.Interface(
    fn=text_to_speech,
    inputs="text",
    outputs="audio",
    title="Uzbek TTS",
    description="O'zbekcha matnni ovozga aylantiruvchi model."
)

# Interfeysni ishga tushirish
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ad540057de541adfd2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
